In [2]:
import numpy as np
import lightgbm as lgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
import time
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from IPython.display import display
import time
from sklearn.metrics import mean_squared_error
from sklearn.externals import joblib


import gc

In [3]:
%load_ext Cython


In [4]:
pd.set_option("display.max_columns", 1000)


In [5]:
log = pd.read_csv("~/environment/log.tsv", sep="\t")
events = pd.read_csv("~/environment/events.tsv", sep="\t")
users = pd.read_csv("~/environment/users.tsv", sep="\t")


In [6]:
print(log.shape)
log.head()

(1452089, 7)


,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN
3,1,125296,2017-06-06 14:01:27,1,NaN,NaN,NaN
4,1,125128,2017-06-06 14:03:57,1,NaN,NaN,NaN


In [10]:
log.loc[:,"user_id_str"] = log["user_id"].apply(lambda x:str(x))
log.loc[:,"event_id_str"] = log["event_id"].apply(lambda x:str(x))
log.loc[:,"log_date"] = pd.to_datetime(log["time_stamp"])
log.loc[:,"log_year"] = log.loc[:,"log_date"].apply(lambda x:str(x.year))
log.loc[:,"log_month"] = log.loc[:,"log_date"].apply(lambda x:str(x.month))
log.loc[:,"log_day"] = log.loc[:,"log_date"].apply(lambda x:str(x.day))
log.loc[:,"log_weekday"] = log.loc[:,"log_date"].apply(lambda x:str(x.dayofweek))
log.loc[:,"log_hour"] = log.loc[:,"log_date"].apply(lambda x:str(x.hour))
log.head()

,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,user_id_str,event_id_str,log_date,log_year,log_month,log_day,log_weekday,log_hour
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0,1,6261,2015-03-19 09:15:50,2015,3,19,3,9
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,1,127600,2017-06-05 12:33:17,2017,6,5,0,12
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,1,127600,2017-06-05 12:37:09,2017,6,5,0,12
3,1,125296,2017-06-06 14:01:27,1,NaN,NaN,NaN,1,125296,2017-06-06 14:01:27,2017,6,6,1,14
4,1,125128,2017-06-06 14:03:57,1,NaN,NaN,NaN,1,125128,2017-06-06 14:03:57,2017,6,6,1,14


In [32]:
data1 = pd.merge(log,events, on="event_id")
data = pd.merge(data1, users, on="user_id")
data = data.rename(columns = {"prefecture_x": "event_prefecture", "prefecture_y":"user_prefecture"})
data.head()

,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,user_id_str,event_id_str,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,event_prefecture,first_published_at,female_price,male_price,interest,age,gender,user_prefecture,created_on
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0,1,6261,27,39.0,27,39.0,2015-03-22 14:00:00,東京都,NaN,4000.0,6500.0,NaN,34,女性,静岡県,2012-06-26
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26
3,1,127600,2017-06-05 12:32:04,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26
4,1,127600,2017-06-06 09:37:45,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26


In [33]:
data.loc[:,"event_date"] = pd.to_datetime(data["event_start_at"])
data.loc[:,"event_year"] = data.loc[:,"event_date"].apply(lambda x:str(x.year))
data.loc[:,"event_month"] = data.loc[:,"event_date"].apply(lambda x:str(x.month))
data.loc[:,"event_day"] = data.loc[:,"event_date"].apply(lambda x:str(x.day))
data.loc[:,"event_weekday"] = data.loc[:,"event_date"].apply(lambda x:str(x.dayofweek))
data.loc[:,"event_hour"] = data.loc[:,"event_date"].apply(lambda x:str(x.hour))

data.head()

,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,user_id_str,event_id_str,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,event_prefecture,first_published_at,female_price,male_price,interest,age,gender,user_prefecture,created_on,event_date,event_year,event_month,event_day,event_weekday,event_hour
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0,1,6261,27,39.0,27,39.0,2015-03-22 14:00:00,東京都,NaN,4000.0,6500.0,NaN,34,女性,静岡県,2012-06-26,2015-03-22 14:00:00,2015,3,22,6,14
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10
3,1,127600,2017-06-05 12:32:04,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10
4,1,127600,2017-06-06 09:37:45,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10


In [34]:
data.loc[:,"user_date"] = pd.to_datetime(data["created_on"])
data.loc[:,"user_year"] = data.loc[:,"user_date"].apply(lambda x:str(x.year))
data.loc[:,"user_month"] = data.loc[:,"user_date"].apply(lambda x:str(x.month))

data.head()

,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,user_id_str,event_id_str,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,event_prefecture,first_published_at,female_price,male_price,interest,age,gender,user_prefecture,created_on,event_date,event_year,event_month,event_day,event_weekday,event_hour,user_date,user_year,user_month
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0,1,6261,27,39.0,27,39.0,2015-03-22 14:00:00,東京都,NaN,4000.0,6500.0,NaN,34,女性,静岡県,2012-06-26,2015-03-22 14:00:00,2015,3,22,6,14,2012-06-26,2012,6
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6
3,1,127600,2017-06-05 12:32:04,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6
4,1,127600,2017-06-06 09:37:45,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6


In [35]:
features = ["female_age_lower","female_age_upper","male_age_lower","male_age_upper","event_prefecture","female_price","male_price","age", "gender", "user_prefecture","event_year", "event_month",
           "event_day","event_weekday", "event_hour", "user_year", "user_month"]
target = ["action_type"]

categorical = ["event_prefecture", "user_gender", "user_prefecture", 'event_year',"event_month","event_day","event_weekday", "event_hour", "user_year", "user_month"]



In [36]:
data = data[features + target]
#naの行消す
data = data.dropna()
data = data.reset_index(drop=True)
print(data.shape)
data.head()

(1442610, 18)


,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_prefecture,female_price,male_price,age,gender,user_prefecture,event_year,event_month,event_day,event_weekday,event_hour,user_year,user_month,action_type
0,27,39.0,27,39.0,東京都,4000.0,6500.0,34,女性,静岡県,2015,3,22,6,14,2012,6,3
1,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6,1
2,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6,1
3,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6,1
4,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6,1


In [ ]:
X = data[features]
print(X.shape)
X.head()

In [38]:
def prefecture_labeling(x):
     #文字列をラベリング
    label_encoder = LabelEncoder()
    labels = ["北海道","青森県","岩手県","宮城県","秋田県", 
"山形県", "福島県", "茨城県","栃木県","群馬県", 
"埼玉県","千葉県","東京都","神奈川県","新潟県", 
"富山県","石川県","福井県","山梨県","長野県", 
"岐阜県","静岡県","愛知県","三重県","滋賀県", 
"京都府","大阪府","兵庫県","奈良県","和歌山県", 
"鳥取県","島根県","岡山県","広島県","山口県", 
"徳島県","香川県","愛媛県","高知県","福岡県", 
"佐賀県","長崎県","熊本県","大分県","宮崎県", 
"鹿児島県","沖縄県",'その他（海外等）']
    label_encoder.fit(labels)
    le =label_encoder.transform(x)
    
    df = pd.DataFrame(le)
    column_name = ["prefecture"]
    df.columns = column_name
    
    return df
    

def gender_labeling(x):
    
    #文字列をラベリング
    label_encoder = LabelEncoder()
    labels = ["女性","男性"]
    label_encoder.fit(labels)
    le = label_encoder.transform(x)
    
    df = pd.DataFrame(le)
    column_name = ["user_gender"]
    df.columns = column_name
    
    return df
    
    

In [39]:
e_pre = prefecture_labeling(X["event_prefecture"])
e_pre.columns = "event_" + e_pre.columns.values
print(e_pre.shape)
e_pre.head()

(1442610, 1)


,event_prefecture
0,27
1,8
2,8
3,8
4,8


In [40]:
u_pre = prefecture_labeling(X["user_prefecture"])
u_pre.columns = "user_" + u_pre.columns.values
print(u_pre.shape)
u_pre.head()

(1442610, 1)


,user_prefecture
0,43
1,43
2,43
3,43
4,43


In [41]:
gen = gender_labeling(X["gender"])
print(gen.shape)
gen.head()

(1442610, 1)


,user_gender
0,0
1,0
2,0
3,0
4,0


In [42]:
temp = X[list(set(features) - set(["user_prefecture", "event_prefecture", "gender"]))]
print(temp.shape)
temp = pd.concat((temp,gen), axis = 1)
temp = pd.concat((temp,e_pre), axis = 1)
temp = pd.concat((temp,u_pre), axis = 1)

X = temp
print(X.shape)
X.head()


(1442610, 14)
(1442610, 17)


,user_month,event_hour,female_age_lower,female_age_upper,age,male_age_upper,event_month,user_year,event_day,female_price,male_price,event_weekday,event_year,male_age_lower,user_gender,event_prefecture,user_prefecture
0,6,14,27,39.0,34,39.0,3,2012,22,4000.0,6500.0,6,2015,27,0,27,43
1,6,10,24,36.0,34,38.0,6,2012,11,2000.0,6000.0,6,2017,26,0,8,43
2,6,10,24,36.0,34,38.0,6,2012,11,2000.0,6000.0,6,2017,26,0,8,43
3,6,10,24,36.0,34,38.0,6,2012,11,2000.0,6000.0,6,2017,26,0,8,43
4,6,10,24,36.0,34,38.0,6,2012,11,2000.0,6000.0,6,2017,26,0,8,43


In [43]:
X = X.astype(np.int64)

In [44]:
y = data.loc[:,target]
print(y.shape)
y.head()

(1442610, 1)


,action_type
0,3
1,1
2,1
3,1
4,1


# 学習

In [93]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

In [94]:
y_train = np.array(y_train).reshape(-1)
y_train = y_train.astype(np.float64)
print(y_train.shape)
print(y_train.dtype)

(1154088,)
float64


In [95]:
y_valid = np.array(y_valid).reshape(-1)
y_valid = y_valid.astype(np.float64)
print(y_valid.shape)
print(y_valid.dtype)

(288522,)
float64


lightGBMのAPI

In [87]:
lgb_train = lgb.Dataset(X_train, y_train, categorical_feature = categorical)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, categorical_feature = categorical)

In [88]:
grid_params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'mean_squared_error'},
        'learning_rate': 0.1,
        'num_leaves': 23,
        'min_data_in_leaf': 1,
        'num_iteration': 10000,
        'verbose': 0
}


In [89]:
grid_gbm = lgb.cv(params, lgb_train,
                  num_boost_round=50,
            early_stopping_rounds=10)

/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


In [90]:
grid_gbm

{'l2-mean': [0.5509174137028386,
  0.5179168663900994,
  0.4910896881217751,
  0.46925162580750684,
  0.4514873097227152,
  0.4370152119811096,
  0.4252709877015367,
  0.4156517159226776,
  0.4078204855654085,
  0.4013512880791087,
  0.39599559240031806,
  0.3916088664232803,
  0.38796791388722535,
  0.38499362019694827,
  0.38248755029073533,
  0.38045848796281134,
  0.3787337498772475,
  0.37729362741508643,
  0.37595959603141926,
  0.3748783935431243,
  0.3739552846911213,
  0.3731425739614175,
  0.3724213117674553,
  0.3717988507988562,
  0.37122718141705346,
  0.370720083989711,
  0.3702896449799452,
  0.36989938378413323,
  0.36951744611073756,
  0.36913065310370335,
  0.36878812334485966,
  0.36846563561178114,
  0.3681435157449562,
  0.367869688602575,
  0.3676248959785573,
  0.36730300319817366,
  0.3670474642471079,
  0.36684686064402217,
  0.3666219373776238,
  0.3664201292304642,
  0.3662399495184928,
  0.366046877942639,
  0.3658927352941147,
  0.36575659401763766,
  0.365

In [76]:
params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'mean_squared_error'},
        'learning_rate': 0.1,
        'num_leaves': 23,
        'min_data_in_leaf': 1,
        'num_iteration': 10000,
        'verbose': 0
}


In [77]:
gbm = lgb.train(params,
            lgb_train,
            num_boost_round=50,
            valid_sets=lgb_eval,
            early_stopping_rounds=10)

/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


[1]	valid_0's l2: 0.55134
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l2: 0.518201
[3]	valid_0's l2: 0.491125
[4]	valid_0's l2: 0.469136
[5]	valid_0's l2: 0.451271
[6]	valid_0's l2: 0.43672
[7]	valid_0's l2: 0.424873
[8]	valid_0's l2: 0.415222
[9]	valid_0's l2: 0.40729
[10]	valid_0's l2: 0.40082
[11]	valid_0's l2: 0.39547
[12]	valid_0's l2: 0.39108
[13]	valid_0's l2: 0.387395
[14]	valid_0's l2: 0.384436
[15]	valid_0's l2: 0.381942
[16]	valid_0's l2: 0.379821
[17]	valid_0's l2: 0.378078
[18]	valid_0's l2: 0.376565
[19]	valid_0's l2: 0.375346
[20]	valid_0's l2: 0.374261
[21]	valid_0's l2: 0.373267
[22]	valid_0's l2: 0.372424
[23]	valid_0's l2: 0.371752
[24]	valid_0's l2: 0.371078
[25]	valid_0's l2: 0.370513
[26]	valid_0's l2: 0.369995
[27]	valid_0's l2: 0.369551
[28]	valid_0's l2: 0.369137
[29]	valid_0's l2: 0.368748
[30]	valid_0's l2: 0.368414
[31]	valid_0's l2: 0.368133
[32]	valid_0's l2: 0.367735
[33]	valid_0's l2: 0.367468
[34]	valid_0's l2: 0.367118
[

KeyboardInterrupt: 

In [ ]:
gbm.save_model('model.txt')


sklearnのAPI

In [115]:
gbm = lgb.LGBMModel(objective ="regression")

In [116]:
gbm.fit(X_train,y_train)

LGBMModel(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
     learning_rate=0.1, max_depth=-1, min_child_samples=20,
     min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
     n_jobs=-1, num_leaves=31, objective='regression', random_state=None,
     reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
     subsample_for_bin=200000, subsample_freq=1)

In [117]:
gbm.best_score_

defaultdict(dict, {})

In [118]:
gbm.best_iteration_

In [119]:
pred = gbm.predict(X_valid)
mean_squared_error(pred, y_valid)

0.36018030984198224

In [120]:
joblib.dump(gbm, 'gbm.pkl') 



['gbm.pkl']

rankを学習させる

In [121]:
gbm = lgb.LGBMModel(objective ="lambdarank")

In [122]:
gbm.fit(X_train,y_train)

LightGBMError: b'Lambdarank tasks require query information'

# predict

In [50]:
gbm = lgb.Booster(model_file='model.txt')



In [28]:
test = pd.read_csv("~/environment/test.tsv", sep="\t")
events = pd.read_csv("~/environment/events.tsv", sep="\t")
users = pd.read_csv("~/environment/users.tsv", sep="\t")


In [29]:
print(users.shape)
users.head()

(69667, 5)


,user_id,age,gender,prefecture,created_on
0,1,34,女性,静岡県,2012-06-26
1,2,31,男性,鳥取県,2012-07-12
2,3,32,男性,東京都,2012-07-15
3,4,30,男性,東京都,2012-07-17
4,5,29,女性,埼玉県,2012-07-17


In [30]:

users.loc[:,"user_date"] = pd.to_datetime(users["created_on"])
users.loc[:,"user_year"] = users.loc[:,"user_date"].apply(lambda x:str(x.year))
users.loc[:,"user_month"] = users.loc[:,"user_date"].apply(lambda x:str(x.month))

print(users.shape)
users.head()

(69667, 8)


,user_id,age,gender,prefecture,created_on,user_date,user_year,user_month
0,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6
1,2,31,男性,鳥取県,2012-07-12,2012-07-12,2012,7
2,3,32,男性,東京都,2012-07-15,2012-07-15,2012,7
3,4,30,男性,東京都,2012-07-17,2012-07-17,2012,7
4,5,29,女性,埼玉県,2012-07-17,2012-07-17,2012,7


In [31]:
#2017-9-24~2017-9-30のログを予想すればよいので，eventを絞る
events.loc[:,"event_date"] = pd.to_datetime(events["event_start_at"])

events = events.loc[events["event_date"] > pd.to_datetime("24/9/2017"),:]
events = events.reset_index(drop = True)
print(events.shape)
events.head()

(2422, 12)


,event_id,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,prefecture,first_published_at,female_price,male_price,interest,event_date
0,143678,32,39.0,36,43.0,2017-09-24 11:00:00,福岡県,2017-08-07 12:29:31,2000.0,3000.0,NaN,2017-09-24 11:00:00
1,143679,39,46.0,43,50.0,2017-09-30 11:00:00,福岡県,2017-08-07 12:48:29,2000.0,3000.0,NaN,2017-09-30 11:00:00
2,143688,37,44.0,41,48.0,2017-09-24 13:30:00,福岡県,2017-08-07 12:34:50,2000.0,3000.0,NaN,2017-09-24 13:30:00
3,143689,32,44.0,36,49.0,2017-09-30 13:30:00,福岡県,2017-08-07 12:53:24,2000.0,3000.0,NaN,2017-09-30 13:30:00
4,143698,26,41.0,30,45.0,2017-09-24 16:00:00,福岡県,2017-08-07 12:40:09,2000.0,3000.0,NaN,2017-09-24 16:00:00


In [32]:

events.loc[:,"event_year"] = events.loc[:,"event_date"].apply(lambda x:str(x.year))
events.loc[:,"event_month"] = events.loc[:,"event_date"].apply(lambda x:str(x.month))
events.loc[:,"event_day"] = events.loc[:,"event_date"].apply(lambda x:str(x.day))
events.loc[:,"event_weekday"] = events.loc[:,"event_date"].apply(lambda x:str(x.dayofweek))
events.loc[:,"event_hour"] = events.loc[:,"event_date"].apply(lambda x:str(x.hour))

print(events.shape)
events.head()

(2422, 17)


,event_id,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,prefecture,first_published_at,female_price,male_price,interest,event_date,event_year,event_month,event_day,event_weekday,event_hour
0,143678,32,39.0,36,43.0,2017-09-24 11:00:00,福岡県,2017-08-07 12:29:31,2000.0,3000.0,NaN,2017-09-24 11:00:00,2017,9,24,6,11
1,143679,39,46.0,43,50.0,2017-09-30 11:00:00,福岡県,2017-08-07 12:48:29,2000.0,3000.0,NaN,2017-09-30 11:00:00,2017,9,30,5,11
2,143688,37,44.0,41,48.0,2017-09-24 13:30:00,福岡県,2017-08-07 12:34:50,2000.0,3000.0,NaN,2017-09-24 13:30:00,2017,9,24,6,13
3,143689,32,44.0,36,49.0,2017-09-30 13:30:00,福岡県,2017-08-07 12:53:24,2000.0,3000.0,NaN,2017-09-30 13:30:00,2017,9,30,5,13
4,143698,26,41.0,30,45.0,2017-09-24 16:00:00,福岡県,2017-08-07 12:40:09,2000.0,3000.0,NaN,2017-09-24 16:00:00,2017,9,24,6,16


In [33]:
e_pre = prefecture_labeling(events["prefecture"])
e_pre.columns = "event_" + e_pre.columns.values
print(e_pre.shape)
e_pre.head()

(2422, 1)


,event_prefecture
0,35
1,35
2,35
3,35
4,35


In [34]:

events = pd.concat( [events,e_pre], axis = 1)
print(events.shape)
events.head()

(2422, 18)


,event_id,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,prefecture,first_published_at,female_price,male_price,interest,event_date,event_year,event_month,event_day,event_weekday,event_hour,event_prefecture
0,143678,32,39.0,36,43.0,2017-09-24 11:00:00,福岡県,2017-08-07 12:29:31,2000.0,3000.0,NaN,2017-09-24 11:00:00,2017,9,24,6,11,35
1,143679,39,46.0,43,50.0,2017-09-30 11:00:00,福岡県,2017-08-07 12:48:29,2000.0,3000.0,NaN,2017-09-30 11:00:00,2017,9,30,5,11,35
2,143688,37,44.0,41,48.0,2017-09-24 13:30:00,福岡県,2017-08-07 12:34:50,2000.0,3000.0,NaN,2017-09-24 13:30:00,2017,9,24,6,13,35
3,143689,32,44.0,36,49.0,2017-09-30 13:30:00,福岡県,2017-08-07 12:53:24,2000.0,3000.0,NaN,2017-09-30 13:30:00,2017,9,30,5,13,35
4,143698,26,41.0,30,45.0,2017-09-24 16:00:00,福岡県,2017-08-07 12:40:09,2000.0,3000.0,NaN,2017-09-24 16:00:00,2017,9,24,6,16,35


In [35]:
u_pre = prefecture_labeling(users["prefecture"])
u_pre.columns = "user_" + u_pre.columns.values
gen = gender_labeling(users["gender"])

users = pd.concat((users, u_pre), axis =1)
users = pd.concat((users, gen), axis = 1)
print(users.shape)
users.head()


(69667, 10)


,user_id,age,gender,prefecture,created_on,user_date,user_year,user_month,user_prefecture,user_gender
0,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6,43,0
1,2,31,男性,鳥取県,2012-07-12,2012-07-12,2012,7,46,1
2,3,32,男性,東京都,2012-07-15,2012-07-15,2012,7,27,1
3,4,30,男性,東京都,2012-07-17,2012-07-17,2012,7,27,1
4,5,29,女性,埼玉県,2012-07-17,2012-07-17,2012,7,8,0


In [36]:
users.loc[users["user_id"] == 69668,:]

,user_id,age,gender,prefecture,created_on,user_date,user_year,user_month,user_prefecture,user_gender


In [37]:
users.loc[69665:,:]

,user_id,age,gender,prefecture,created_on,user_date,user_year,user_month,user_prefecture,user_gender
69665,69666,23,女性,東京都,2017-09-23,2017-09-23,2017,9,27,0
69666,69667,42,男性,千葉県,2017-09-23,2017-09-23,2017,9,6,1


In [38]:
user_event = events
user_event.loc[:,"user_id"] = 1

user_data = pd.merge(events, users, on = "user_id")

print(user_data.shape)
user_data.head()

(2422, 28)


,event_id,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,prefecture_x,first_published_at,female_price,male_price,interest,event_date,event_year,event_month,event_day,event_weekday,event_hour,event_prefecture,user_id,age,gender,prefecture_y,created_on,user_date,user_year,user_month,user_prefecture,user_gender
0,143678,32,39.0,36,43.0,2017-09-24 11:00:00,福岡県,2017-08-07 12:29:31,2000.0,3000.0,NaN,2017-09-24 11:00:00,2017,9,24,6,11,35,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6,43,0
1,143679,39,46.0,43,50.0,2017-09-30 11:00:00,福岡県,2017-08-07 12:48:29,2000.0,3000.0,NaN,2017-09-30 11:00:00,2017,9,30,5,11,35,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6,43,0
2,143688,37,44.0,41,48.0,2017-09-24 13:30:00,福岡県,2017-08-07 12:34:50,2000.0,3000.0,NaN,2017-09-24 13:30:00,2017,9,24,6,13,35,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6,43,0
3,143689,32,44.0,36,49.0,2017-09-30 13:30:00,福岡県,2017-08-07 12:53:24,2000.0,3000.0,NaN,2017-09-30 13:30:00,2017,9,30,5,13,35,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6,43,0
4,143698,26,41.0,30,45.0,2017-09-24 16:00:00,福岡県,2017-08-07 12:40:09,2000.0,3000.0,NaN,2017-09-24 16:00:00,2017,9,24,6,16,35,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6,43,0


In [39]:
features = ["female_age_lower","female_age_upper","male_age_lower","male_age_upper","event_prefecture","female_price","male_price","age", "user_gender", "user_prefecture","event_year", "event_month",
           "event_day","event_weekday", "event_hour", "user_year", "user_month"]
target = ["action_type"]

categorical = ["event_prefecture", "user_gender", "user_prefecture", 'event_year',"event_month","event_day","event_weekday", "event_hour", "user_year", "user_month"]


In [40]:


X = user_data[features]
X.loc[:,categorical] = X.loc[:,categorical].astype(np.int64)
print(X.shape)
X.head()

/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(2422, 17)


,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_prefecture,female_price,male_price,age,user_gender,user_prefecture,event_year,event_month,event_day,event_weekday,event_hour,user_year,user_month
0,32,39.0,36,43.0,35,2000.0,3000.0,34,0,43,2017,9,24,6,11,2012,6
1,39,46.0,43,50.0,35,2000.0,3000.0,34,0,43,2017,9,30,5,11,2012,6
2,37,44.0,41,48.0,35,2000.0,3000.0,34,0,43,2017,9,24,6,13,2012,6
3,32,44.0,36,49.0,35,2000.0,3000.0,34,0,43,2017,9,30,5,13,2012,6
4,26,41.0,30,45.0,35,2000.0,3000.0,34,0,43,2017,9,24,6,16,2012,6


In [41]:
y_pred = gbm.predict(X, num_iteration=gbm.best_iteration)


In [42]:
y_pred

array([2.52570494, 2.41102264, 2.35508459, ..., 2.51021086, 2.43699561,
       2.48428178])

In [43]:

user_list = test["user_id"]
event_list = events["event_id"]
print(len(user_list))
print(len(event_list))

2886
2422


In [44]:
###!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!user_id 69667以降はデモグラフィデータが存在しない!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
display(users.tail())
display(user_list.loc[user_list > 69667])
print(user_list.loc[user_list > 69667].shape)

##とりあえずデモグラフィデータが存在するユーザだけに対して予測する
user_list = user_list.loc[user_list < 69667]
display(user_list.tail())
print(user_list.shape)

,user_id,age,gender,prefecture,created_on,user_date,user_year,user_month,user_prefecture,user_gender
69662,69663,24,女性,埼玉県,2017-09-23,2017-09-23,2017,9,8,0
69663,69664,23,女性,宮城県,2017-09-23,2017-09-23,2017,9,12,0
69664,69665,32,男性,岡山県,2017-09-23,2017-09-23,2017,9,19,1
69665,69666,23,女性,東京都,2017-09-23,2017-09-23,2017,9,27,0
69666,69667,42,男性,千葉県,2017-09-23,2017-09-23,2017,9,6,1


2556    69668
2557    69671
2558    69672
2559    69673
2560    69674
2561    69675
2562    69676
2563    69678
2564    69679
2565    69680
2566    69681
2567    69682
2568    69683
2569    69684
2570    69685
2571    69686
2572    69687
2573    69688
2574    69689
2575    69690
2576    69691
2577    69692
2578    69693
2579    69694
2580    69695
2581    69696
2582    69697
2583    69698
2584    69699
2585    69700
        ...  
2856    69971
2857    69972
2858    69973
2859    69974
2860    69975
2861    69976
2862    69977
2863    69978
2864    69979
2865    69980
2866    69981
2867    69982
2868    69983
2869    69984
2870    69985
2871    69986
2872    69987
2873    69988
2874    69989
2875    69990
2876    69991
2877    69992
2878    69993
2879    69994
2880    69995
2881    69996
2882    69997
2883    69998
2884    69999
2885    70000
Name: user_id, Length: 330, dtype: int64

(330,)


2551    69649
2552    69654
2553    69660
2554    69663
2555    69664
Name: user_id, dtype: int64

(2556,)


In [45]:
result = pd.DataFrame({"event_id":event_list})
result.loc[:,"predict"] = y_pred
result = result.sort_values(by=["predict"], ascending=False)
result = result.reset_index(drop = True)
result.loc[:,"rank"] = result.index.values + 1

In [46]:
print(result.shape)
result.head()

(2422, 3)


,event_id,predict,rank
0,153286,2.945760,1
1,155340,2.835655,2
2,157672,2.810693,3
3,154477,2.810693,4
4,157009,2.810693,5


In [47]:
def predict_user(user_id):
    
    user_event.loc[:,"user_id"] = user_id

    user_data = pd.merge(events, users, on = "user_id")
    
    X = user_data[features]
    X.loc[:,categorical] = X.loc[:,categorical].astype(np.int64)
    
    y_pred = gbm.predict(X, num_iteration=gbm.best_iteration)
    
    result = pd.DataFrame({"event_id":event_list})
    
    #print(user_data.shape)
    ##print(X.shape)
    #print(y_pred.shape)
    #print(result.shape)
    #display(result.head())
    
    result.loc[:,"predict"] = y_pred
    result.loc[:,"user_id"] = user_id

    result = result.sort_values(by=["predict"], ascending=False)
    result = result.reset_index(drop = True)
    result.loc[:,"rank"] = result.index.values + 1
    
    return result


In [48]:
start = time.time() #時間計測

predict = predict_user(6)

elapsed_time = time.time() - start #時間計測ここまで

print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


elapsed_time:0.7403678894042969[sec]


In [49]:
predict.head(20)

,event_id,predict,user_id,rank
0,158384,3.252743,6,1
1,144865,3.099696,6,2
2,144843,3.094880,6,3
3,145490,3.068952,6,4
4,158385,3.052659,6,5
5,144840,3.040060,6,6
6,145451,3.040060,6,7
7,145491,3.027771,6,8
8,151358,3.027771,6,9
9,150581,3.027771,6,10


In [50]:


k = 20
i = 0
recommends = []

for user in user_list:
    
    start = time.time() #時間計測

    
    i = i +1
    print("user_id:"+str(user))
    print("進捗:"+str(i*100/len(user_list))+"%") 
    
    result = predict_user(user)
    
    recommend_by_user = result.loc[:k-1,:]
    recommends.append(recommend_by_user)
    
    elapsed_time = time.time() - start #時間計測ここまで
    print ("予想残り時間")
    print(elapsed_time * (len(user_list)-i))


user_id:1
進捗:0.03912363067292645%


/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


予想残り時間
2086.463166475296
user_id:6
進捗:0.0782472613458529%
予想残り時間
2039.3240957260132
user_id:23
進捗:0.11737089201877934%
予想残り時間
2019.3569798469543
user_id:43
進捗:0.1564945226917058%
予想残り時間
2065.834819793701
user_id:46
進捗:0.19561815336463223%
予想残り時間
2156.552997112274
user_id:48
進捗:0.2347417840375587%
予想残り時間
2098.561728000641
user_id:76
進捗:0.27386541471048514%
予想残り時間
2063.3783750534058
user_id:79
進捗:0.3129890453834116%
予想残り時間
2184.5007457733154
user_id:104
進捗:0.352112676056338%
予想残り時間
2467.980710506439
user_id:139
進捗:0.39123630672926446%
予想残り時間
2208.068187713623
user_id:152
進捗:0.4303599374021909%
予想残り時間
2025.9838390350342
user_id:198
進捗:0.4694835680751174%
予想残り時間
2037.3676414489746
user_id:206
進捗:0.5086071987480438%
予想残り時間
2081.3522367477417
user_id:233
進捗:0.5477308294209703%
予想残り時間
2119.193736553192
user_id:242
進捗:0.5868544600938967%
予想残り時間
2068.3912925720215
user_id:247
進捗:0.6259780907668232%
予想残り時間
2113.4502124786377
user_id:263
進捗:0.6651017214397497%
予想残り時間
2183.257037639618
user_id:285

予想残り時間
1949.2110736370087
user_id:2674
進捗:5.39906103286385%
予想残り時間
2036.1255083084106
user_id:2711
進捗:5.438184663536776%
予想残り時間
1988.4999563694
user_id:2777
進捗:5.477308294209703%
予想残り時間
2063.552806854248
user_id:2803
進捗:5.516431924882629%
予想残り時間
2082.228022813797
user_id:2899
進捗:5.555555555555555%
予想残り時間
2074.5473651885986
user_id:2902
進捗:5.594679186228482%
予想残り時間
2023.8419163227081
user_id:2904
進捗:5.633802816901408%
予想残り時間
2041.6841983795166
user_id:2908
進捗:5.672926447574335%
予想残り時間
1934.602190732956
user_id:2987
進捗:5.712050078247262%
予想残り時間
2009.122052192688
user_id:2997
進捗:5.751173708920188%
予想残り時間
1975.6285328865051
user_id:3011
進捗:5.790297339593114%
予想残り時間
1939.6779460906982
user_id:3031
進捗:5.829420970266041%
予想残り時間
1989.4088819026947
user_id:3039
進捗:5.868544600938967%
予想残り時間
1966.938546180725
user_id:3050
進捗:5.907668231611893%
予想残り時間
1961.1416554450989
user_id:3062
進捗:5.94679186228482%
予想残り時間
2016.0022325515747
user_id:3063
進捗:5.985915492957746%
予想残り時間
1959.4832682609558
user_id:

予想残り時間
1829.0998191833496
user_id:5410
進捗:10.68075117370892%
予想残り時間
1833.9015662670135
user_id:5432
進捗:10.719874804381847%
予想残り時間
1807.3490228652954
user_id:5434
進捗:10.758998435054773%
予想残り時間
1814.5899746418
user_id:5500
進捗:10.7981220657277%
予想残り時間
1852.3135471343994
user_id:5509
進捗:10.837245696400625%
予想残り時間
1784.1516165733337
user_id:5511
進捗:10.876369327073553%
予想残り時間
1865.2105870246887
user_id:5546
進捗:10.915492957746478%
予想残り時間
1785.7943000793457
user_id:5592
進捗:10.954616588419405%
予想残り時間
1758.183497428894
user_id:5614
進捗:10.993740219092333%
予想残り時間
1869.9171721935272
user_id:5623
進捗:11.032863849765258%
予想残り時間
1889.9251656532288
user_id:5650
進捗:11.071987480438185%
予想残り時間
1760.5200879573822
user_id:5655
進捗:11.11111111111111%
予想残り時間
1874.6525344848633
user_id:5660
進捗:11.150234741784038%
予想残り時間
1869.6988778114319
user_id:5738
進捗:11.189358372456963%
予想残り時間
1883.7431693077087
user_id:5743
進捗:11.22848200312989%
予想残り時間
1922.3020210266113
user_id:5745
進捗:11.267605633802816%
予想残り時間
1808.63026

...

予想残り時間
489.9496250152588
user_id:57553
進捗:78.67762128325509%
予想残り時間
499.57194209098816
user_id:57560
進捗:78.71674491392801%
予想残り時間
482.4434814453125
user_id:57572
進捗:78.75586854460094%
予想残り時間
525.0231349468231
user_id:57778
進捗:78.79499217527386%
予想残り時間
530.6320056915283
user_id:57861
進捗:78.8341158059468%
予想残り時間
554.1678936481476
user_id:57922
進捗:78.87323943661971%
予想残り時間
493.97393703460693
user_id:57954
進捗:78.91236306729265%
予想残り時間
516.7161371707916
user_id:57960
進捗:78.95148669796558%
予想残り時間
556.9852519035339
user_id:57964
進捗:78.9906103286385%
予想残り時間
503.62596130371094
user_id:58052
進捗:79.02973395931143%
予想残り時間
501.5750389099121
user_id:58061
進捗:79.06885758998435%
予想残り時間
496.5297317504883
user_id:58079
進捗:79.10798122065728%
予想残り時間
525.6396989822388
user_id:58084
進捗:79.1471048513302%
予想残り時間
501.9713296890259
user_id:58088
進捗:79.18622848200313%
予想残り時間
509.29689025878906
user_id:58109
進捗:79.22535211267606%
予想残り時間
508.38931918144226
user_id:58112
進捗:79.26447574334898%
予想残り時間
493.26989650726

予想残り時間
380.3601236343384
user_id:62089
進捗:83.92018779342723%
予想残り時間
387.16186594963074
user_id:62161
進捗:83.95931142410015%
予想残り時間
411.74535512924194
user_id:62193
進捗:83.99843505477308%
予想残り時間
380.26451802253723
user_id:62194
進捗:84.03755868544602%
予想残り時間
404.1144561767578
user_id:62242
進捗:84.07668231611893%
予想残り時間
387.2278709411621
user_id:62312
進捗:84.11580594679187%
予想残り時間
402.0839424133301
user_id:62428
進捗:84.15492957746478%
予想残り時間
388.19931864738464
user_id:62431
進捗:84.19405320813772%
予想残り時間
384.01577854156494
user_id:62469
進捗:84.23317683881064%
予想残り時間
374.10188269615173
user_id:62525
進捗:84.27230046948357%
予想残り時間
374.0308213233948
user_id:62534
進捗:84.3114241001565%
予想残り時間
392.3834271430969
user_id:62544
進捗:84.35054773082942%
予想残り時間
375.0908851623535
user_id:62583
進捗:84.38967136150235%
予想残り時間
368.4937427043915
user_id:62592
進捗:84.42879499217527%
予想残り時間
372.76033544540405
user_id:62646
進捗:84.4679186228482%
予想残り時間
387.42896127700806
user_id:62653
進捗:84.50704225352112%
予想残り時間
417.1938114

予想残り時間
261.1345634460449
user_id:65733
進捗:89.16275430359937%
予想残り時間
257.3862683773041
user_id:65792
進捗:89.2018779342723%
予想残り時間
270.0016794204712
user_id:65829
進捗:89.24100156494522%
予想残り時間
272.9227662086487
user_id:65844
進捗:89.28012519561815%
予想残り時間
280.9757800102234
user_id:65871
進捗:89.31924882629107%
予想残り時間
255.4044270515442
user_id:65875
進捗:89.358372456964%
予想残り時間
262.8479995727539
user_id:65879
進捗:89.39749608763694%
予想残り時間
265.39592719078064
user_id:65885
進捗:89.43661971830986%
予想残り時間
299.3414783477783
user_id:65892
進捗:89.47574334898279%
予想残り時間
258.3412787914276
user_id:65947
進捗:89.51486697965571%
予想残り時間
246.37254905700684
user_id:65981
進捗:89.55399061032864%
予想残り時間
264.5483486652374
user_id:65982
進捗:89.59311424100156%
予想残り時間
244.14005041122437
user_id:65989
進捗:89.63223787167449%
予想残り時間
242.84099698066711
user_id:66010
進捗:89.67136150234742%
予想残り時間
263.12969398498535
user_id:66032
進捗:89.71048513302034%
予想残り時間
246.95991849899292
user_id:66034
進捗:89.74960876369327%
予想残り時間
251.2472848892

予想残り時間
137.81239700317383
user_id:68116
進捗:94.40532081377152%
予想残り時間
136.97431993484497
user_id:68126
進捗:94.44444444444444%
予想残り時間
137.11144399642944
user_id:68146
進捗:94.48356807511738%
予想残り時間
132.1434953212738
user_id:68172
進捗:94.5226917057903%
予想残り時間
133.8353395462036
user_id:68194
進捗:94.56181533646323%
予想残り時間
133.14903450012207
user_id:68220
進捗:94.60093896713614%
予想残り時間
130.6946897506714
user_id:68222
進捗:94.64006259780908%
予想残り時間
143.10905814170837
user_id:68232
進捗:94.67918622848201%
予想残り時間
129.83845329284668
user_id:68236
進捗:94.71830985915493%
予想残り時間
134.89439606666565
user_id:68323
進捗:94.75743348982786%
予想残り時間
131.9876856803894
user_id:68329
進捗:94.79655712050078%
予想残り時間
133.2729251384735
user_id:68331
進捗:94.83568075117371%
予想残り時間
141.3017692565918
user_id:68337
進捗:94.87480438184663%
予想残り時間
138.8077712059021
user_id:68340
進捗:94.91392801251956%
予想残り時間
126.34278297424316
user_id:68401
進捗:94.9530516431925%
予想残り時間
128.87211656570435
user_id:68402
進捗:94.99217527386541%
予想残り時間
125.941589

予想残り時間
10.520577430725098
user_id:69616
進捗:99.64788732394366%
予想残り時間
9.263437986373901
user_id:69620
進捗:99.68701095461658%
予想残り時間
7.978879928588867
user_id:69621
進捗:99.72613458528951%
予想残り時間
7.068880081176758
user_id:69646
進捗:99.76525821596245%
予想残り時間
6.4063639640808105
user_id:69647
進捗:99.80438184663537%
予想残り時間
4.8628973960876465
user_id:69649
進捗:99.8435054773083%
予想残り時間
4.047069549560547
user_id:69654
進捗:99.88262910798122%
予想残り時間
2.961355447769165
user_id:69660
進捗:99.92175273865415%
予想残り時間
1.9304027557373047
user_id:69663
進捗:99.96087636932707%
予想残り時間
0.9822726249694824
user_id:69664
進捗:100.0%
予想残り時間
0.0


In [51]:
recommends

[    event_id   predict  user_id  rank
 0     153286  2.945760        1     1
 1     155340  2.835655        1     2
 2     157672  2.810693        1     3
 3     154477  2.810693        1     4
 4     157009  2.810693        1     5
 5     147098  2.810693        1     6
 6     152533  2.780549        1     7
 7     148057  2.780549        1     8
 8     158329  2.780549        1     9
 9     156312  2.780549        1    10
 10    152844  2.780549        1    11
 11    152831  2.780549        1    12
 12    146344  2.780549        1    13
 13    150590  2.780549        1    14
 14    157458  2.780549        1    15
 15    155476  2.780549        1    16
 16    152769  2.780549        1    17
 17    155765  2.780549        1    18
 18    152753  2.780549        1    19
 19    147411  2.780549        1    20,     event_id   predict  user_id  rank
 0     158384  3.252743        6     1
 1     144865  3.099696        6     2
 2     144843  3.094880        6     3
 3     145490  3.068952  

In [52]:
f = open('recommend_lightGBM.txt', 'w')
for x in recommends:
    f.write(str(x) + "\n")
f.close()

In [53]:
sub = pd.DataFrame()

for recommend in recommends:
    
    sub = pd.concat([sub,recommend])
    

In [54]:
sub.head(50)

,event_id,predict,user_id,rank
0,153286,2.945760,1,1
1,155340,2.835655,1,2
2,157672,2.810693,1,3
3,154477,2.810693,1,4
4,157009,2.810693,1,5
5,147098,2.810693,1,6
6,152533,2.780549,1,7
7,148057,2.780549,1,8
8,158329,2.780549,1,9
9,156312,2.780549,1,10


In [55]:
sub.tail()

,event_id,predict,user_id,rank
15,153286,3.037584,69664,16
16,147944,3.034443,69664,17
17,145491,3.031421,69664,18
18,150581,3.031421,69664,19
19,151358,3.031421,69664,20


In [56]:
sub = sub.loc[:,["user_id","event_id","rank"]]
print(sub.shape)
sub.head(50)

(51120, 3)


,user_id,event_id,rank
0,1,153286,1
1,1,155340,2
2,1,157672,3
3,1,154477,4
4,1,157009,5
5,1,147098,6
6,1,152533,7
7,1,148057,8
8,1,158329,9
9,1,156312,10


In [60]:
sample_submit = pd.read_csv("~/environment/sample_submit.tsv", sep="\t", header = None)
sample_submit.columns = sub.columns
print(sample_submit.shape)
sample_submit.head()

(57720, 3)


,user_id,event_id,rank
0,1,1,1
1,1,2,2
2,1,3,3
3,1,4,4
4,1,5,5


In [63]:
submmition = pd.concat([sub,sample_submit.loc[sample_submit["user_id"]> 69667,:]])
print(submmition.shape)
submmition.head()

(57720, 3)


,user_id,event_id,rank
0,1,153286,1
1,1,155340,2
2,1,157672,3
3,1,154477,4
4,1,157009,5


In [64]:
submmition.to_csv("sub_lightgbm.tsv",header=None,index=None,sep = "\t")

# 以下工事中

# categorycaなfeatureをダミー化

しなくてよかったっぽい
https://lightgbm.readthedocs.io/en/latest/Python-Intro.html

In [46]:
#categoricalな特徴をダミー化する
#各特徴に専用の関数を作る

def month_dummying(x):
    onehot_encoder = OneHotEncoder()
    
    cat = [1,2,3,4,5,6,7,8,9,10,11,12]
    
    category = np.array(cat).reshape(-1,1)
    
    onehot_encoder.fit(category)
    
    mat =  onehot_encoder.transform(np.array([x]).reshape(-1,1)).toarray()
    
    df = pd.DataFrame(mat)
    
    column_name = list(map(lambda x:"month"+str(x),cat))
    
    df.columns = column_name
    
    return df
    
def day_dummying(x):
    onehot_encoder = OneHotEncoder()
    
    cat = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]
    
    category = np.array(cat).reshape(-1,1)
    
    onehot_encoder.fit(category)
    
    mat =  onehot_encoder.transform(np.array([x]).reshape(-1,1)).toarray()
    
    df = pd.DataFrame(mat)
    
    column_name = list(map(lambda x:"day"+str(x),cat))
    
    df.columns = column_name
    
    return df

def year_dummying(x):
    
    onehot_encoder = OneHotEncoder()
    
    cat = [2012,2013,2014,2015,2016,2017]
    
    category = np.array(cat).reshape(-1,1)
    
    onehot_encoder.fit(category)
    
    mat =  onehot_encoder.transform(np.array([x]).reshape(-1,1)).toarray()
    
    df = pd.DataFrame(mat)
    
    column_name = list(map(lambda x:"year"+str(x),cat))
    
    df.columns = column_name
    
    return df
    

def weekday_dummying(x):
    
    onehot_encoder = OneHotEncoder()
    
    cat = [0,1,2,3,4,5,6]
    
    category = np.array(cat).reshape(-1,1)
    
    onehot_encoder.fit(category)
    
    mat =  onehot_encoder.transform(np.array([x]).reshape(-1,1)).toarray()
    
    df = pd.DataFrame(mat)
    
    column_name = list(map(lambda x:"week_day"+str(x),cat))
    
    df.columns = column_name
    
    return df

def hour_dummying(x):
    
    onehot_encoder = OneHotEncoder()
    
    cat = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]
    
    category = np.array(cat).reshape(-1,1)
    
    onehot_encoder.fit(category)
    
    mat =  onehot_encoder.transform(np.array([x]).reshape(-1,1)).toarray()
    
    df = pd.DataFrame(mat)
    
    column_name = list(map(lambda x:"hour"+str(x),cat))
    
    df.columns = column_name
    
    return df
    
    
    

In [47]:
def gender_dummying(x):
    
    #文字列をラベリング
    label_encoder = LabelEncoder()
    labels = ["女性","男性"]
    label_encoder.fit(labels)
    le = label_encoder.transform(x)
    
    #数値をonehot vectorへ
    
    onehot_encoder = OneHotEncoder()
    category = np.array(np.arange(len(labels)).reshape(-1,1))
    onehot_encoder.fit(category)
    
    mat = onehot_encoder.transform(np.array([le]).reshape(-1,1)).toarray()
    df = pd.DataFrame(mat)
    column_name = label_encoder.classes_
    df.columns = column_name
    
    return df
    

def prefecture_dummying(x):
    
    #文字列をラベリング
    label_encoder = LabelEncoder()
    labels = ["北海道","青森県","岩手県","宮城県","秋田県", 
"山形県", "福島県", "茨城県","栃木県","群馬県", 
"埼玉県","千葉県","東京都","神奈川県","新潟県", 
"富山県","石川県","福井県","山梨県","長野県", 
"岐阜県","静岡県","愛知県","三重県","滋賀県", 
"京都府","大阪府","兵庫県","奈良県","和歌山県", 
"鳥取県","島根県","岡山県","広島県","山口県", 
"徳島県","香川県","愛媛県","高知県","福岡県", 
"佐賀県","長崎県","熊本県","大分県","宮崎県", 
"鹿児島県","沖縄県",'その他（海外等）']
    label_encoder.fit(labels)
    le =label_encoder.transform(x)
    
    label_encoder.classes_
    
    #数値をonehot vectorへ
    
    onehot_encoder = OneHotEncoder()
    category = np.array(np.arange(len(labels))).reshape(-1,1)
    onehot_encoder.fit(category)
    
    mat = onehot_encoder.transform(np.array([le]).reshape(-1,1)).toarray()
    df = pd.DataFrame(mat)
    column_name = label_encoder.classes_
    df.columns = column_name
    
    return df

categorycalな列を変換

In [48]:
e_pre = prefecture_dummying(X["event_prefercture"])
e_pre.columns = "e_" + e_pre.columns.values
print(e_pre.shape)
e_pre.head()

KeyError: 'event_prefercture'

In [49]:
u_pre = prefecture_dummying(X["user_prefecture"])
u_pre.columns = "u_" + u_pre.columns.values
print(u_pre.shape)
u_pre.head()

KeyError: 'user_prefecture'

In [50]:
gender = gender_dummying(X["gender"])
print(gender.shape)
gender.head()

KeyError: 'gender'

In [16]:
u_year = year_dummying(X["user_year"])
u_year.columns = "u_" + u_year.columns.values
print(u_year.shape)
u_year.head()


(1450405, 6)


,u_year2012,u_year2013,u_year2014,u_year2015,u_year2016,u_year2017
0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0


In [17]:
u_month = month_dummying(X["user_month"])
u_month.columns = "u_" + u_month.columns.values
print(u_month.shape)
u_month.head()

(1450405, 12)


,u_month1,u_month2,u_month3,u_month4,u_month5,u_month6,u_month7,u_month8,u_month9,u_month10,u_month11,u_month12
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
e_year = year_dummying(X["event_year"])
e_year.columns = "e_" + e_year.columns.values
print(e_year.shape)
e_year.head()


(1450405, 6)


,e_year2012,e_year2013,e_year2014,e_year2015,e_year2016,e_year2017
0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,1.0


In [19]:
e_month = month_dummying(X["event_month"])
e_month.columns = "e_" + e_month.columns.values
print(e_month.shape)
e_month.head()


(1450405, 12)


,e_month1,e_month2,e_month3,e_month4,e_month5,e_month6,e_month7,e_month8,e_month9,e_month10,e_month11,e_month12
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
e_day = day_dummying(X["event_day"])
e_day.columns = "e_" + e_day.columns.values
print(e_day.shape)
e_day.head()

(1450405, 31)


,e_day1,e_day2,e_day3,e_day4,e_day5,e_day6,e_day7,e_day8,e_day9,e_day10,e_day11,e_day12,e_day13,e_day14,e_day15,e_day16,e_day17,e_day18,e_day19,e_day20,e_day21,e_day22,e_day23,e_day24,e_day25,e_day26,e_day27,e_day28,e_day29,e_day30,e_day31
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
e_weekday = weekday_dummying(X["event_weekday"])
e_weekday.columns = "e_" + e_weekday.columns.values
print(e_weekday.shape)
e_weekday.head()

(1450405, 7)


,e_week_day0,e_week_day1,e_week_day2,e_week_day3,e_week_day4,e_week_day5,e_week_day6
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [22]:
e_hour = hour_dummying(X["event_hour"])
e_hour.columns = "e_" + e_hour.columns.values
print(e_hour.shape)
e_hour.head()

(1450405, 25)


,e_hour0,e_hour1,e_hour2,e_hour3,e_hour4,e_hour5,e_hour6,e_hour7,e_hour8,e_hour9,e_hour10,e_hour11,e_hour12,e_hour13,e_hour14,e_hour15,e_hour16,e_hour17,e_hour18,e_hour19,e_hour20,e_hour21,e_hour22,e_hour23,e_hour24
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
temp = X[list(set(features) - set(categorical))]
temp = pd.concat((temp,e_pre), axis = 1)
temp = pd.concat((temp,gender), axis  =1 )
temp = pd.concat((temp,u_pre), axis = 1)
temp = pd.concat((temp, e_year), axis = 1)
temp = pd.concat((temp, e_month), axis = 1)
temp = pd.concat((temp, e_day), axis = 1)
temp = pd.concat((temp, e_weekday), axis = 1)
temp = pd.concat((temp, e_hour), axis = 1)
temp = pd.concat((temp, u_year), axis = 1)
temp = pd.concat((temp, u_month), axis = 1)
print(temp.shape)
temp.head()

(1450405, 204)


,age,female_price,male_age_upper,female_age_upper,male_age_lower,female_age_lower,male_price,e_その他（海外等）,e_三重県,e_京都府,e_佐賀県,e_兵庫県,e_北海道,e_千葉県,e_和歌山県,e_埼玉県,e_大分県,e_大阪府,e_奈良県,e_宮城県,e_宮崎県,e_富山県,e_山口県,e_山形県,e_山梨県,e_岐阜県,e_岡山県,e_岩手県,e_島根県,e_広島県,e_徳島県,e_愛媛県,e_愛知県,e_新潟県,e_東京都,e_栃木県,e_沖縄県,e_滋賀県,e_熊本県,e_石川県,e_神奈川県,e_福井県,e_福岡県,e_福島県,e_秋田県,e_群馬県,e_茨城県,e_長崎県,e_長野県,e_青森県,e_静岡県,e_香川県,e_高知県,e_鳥取県,e_鹿児島県,女性,男性,u_その他（海外等）,u_三重県,u_京都府,u_佐賀県,u_兵庫県,u_北海道,u_千葉県,u_和歌山県,u_埼玉県,u_大分県,u_大阪府,u_奈良県,u_宮城県,u_宮崎県,u_富山県,u_山口県,u_山形県,u_山梨県,u_岐阜県,u_岡山県,u_岩手県,u_島根県,u_広島県,u_徳島県,u_愛媛県,u_愛知県,u_新潟県,u_東京都,u_栃木県,u_沖縄県,u_滋賀県,u_熊本県,u_石川県,u_神奈川県,u_福井県,u_福岡県,u_福島県,u_秋田県,u_群馬県,u_茨城県,u_長崎県,u_長野県,u_青森県,u_静岡県,u_香川県,u_高知県,u_鳥取県,u_鹿児島県,e_year2012,e_year2013,e_year2014,e_year2015,e_year2016,e_year2017,e_month1,e_month2,e_month3,e_month4,e_month5,e_month6,e_month7,e_month8,e_month9,e_month10,e_month11,e_month12,e_day1,e_day2,e_day3,e_day4,e_day5,e_day6,e_day7,e_day8,e_day9,e_day10,e_day11,e_day12,e_day13,e_day14,e_day15,e_day16,e_day17,e_day18,e_day19,e_day20,e_day21,e_day22,e_day23,e_day24,e_day25,e_day26,e_day27,e_day28,e_day29,e_day30,e_day31,e_week_day0,e_week_day1,e_week_day2,e_week_day3,e_week_day4,e_week_day5,e_week_day6,e_hour0,e_hour1,e_hour2,e_hour3,e_hour4,e_hour5,e_hour6,e_hour7,e_hour8,e_hour9,e_hour10,e_hour11,e_hour12,e_hour13,e_hour14,e_hour15,e_hour16,e_hour17,e_hour18,e_hour19,e_hour20,e_hour21,e_hour22,e_hour23,e_hour24,u_year2012,u_year2013,u_year2014,u_year2015,u_year2016,u_year2017,u_month1,u_month2,u_month3,u_month4,u_month5,u_month6,u_month7,u_month8,u_month9,u_month10,u_month11,u_month12
0,34,4000.0,39.0,39.0,27,27,6500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,34,2000.0,38.0,36.0,26,24,6000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,34,2000.0,38.0,36.0,26,24,6000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,

# 学習

In [25]:
X = temp

In [26]:
y = data.loc[:,target]
print(y.shape)
y.head()

(1450405, 1)


,action_type
0,3
1,1
2,1
3,1
4,1


In [27]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

In [28]:
y_train = np.array(y_train).reshape(-1)
y_train = y_train.astype(np.float64)
print(y_train.shape)
print(y_train.dtype)

(1160324,)
float64


In [29]:
y_valid = np.array(y_valid).reshape(-1)
y_valid = y_valid.astype(np.float64)
print(y_valid.shape)
print(y_valid.dtype)

(290081,)
float64


In [48]:
lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=['c3'])
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

In [55]:
lgb_train.get_field("gender")

Exception: Cannot get gender before construct Dataset

In [38]:
params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'mean_squared_error'},
        'learning_rate': 0.1,
        'num_leaves': 23,
        'min_data_in_leaf': 1,
        'num_iteration': 10000,
        'verbose': 0
}


In [39]:
gbm = lgb.train(params,
            lgb_train,
            num_boost_round=50,
            valid_sets=lgb_eval,
            early_stopping_rounds=10)


/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l2: 0.552014
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l2: 0.519943
[3]	valid_0's l2: 0.493896
[4]	valid_0's l2: 0.47265
[5]	valid_0's l2: 0.455379
[6]	valid_0's l2: 0.44132
[7]	valid_0's l2: 0.429646
[8]	valid_0's l2: 0.420306
[9]	valid_0's l2: 0.412629
[10]	valid_0's l2: 0.406207
[11]	valid_0's l2: 0.401102
[12]	valid_0's l2: 0.39677
[13]	valid_0's l2: 0.393205
[14]	valid_0's l2: 0.390245
[15]	valid_0's l2: 0.387918
[16]	valid_0's l2: 0.385837
[17]	valid_0's l2: 0.384236
[18]	valid_0's l2: 0.382449
[19]	valid_0's l2: 0.381205
[20]	valid_0's l2: 0.379905
[21]	valid_0's l2: 0.379007
[22]	valid_0's l2: 0.377982
[23]	valid_0's l2: 0.377311
[24]	valid_0's l2: 0.376618
[25]	valid_0's l2: 0.376054
[26]	valid_0's l2: 0.375384
[27]	valid_0's l2: 0.374938
[28]	valid_0's l2: 0.374442
[29]	valid_0's l2: 0.374046
[30]	valid_0's l2: 0.373613
[31]	valid_0's l2: 0.373235
[32]	valid_0's l2: 0.372919
[33]	valid_0's l2: 0.372652
[34]	valid_0's l2: 0.37239

[292]	valid_0's l2: 0.357452
[293]	valid_0's l2: 0.357413
[294]	valid_0's l2: 0.357363
[295]	valid_0's l2: 0.357342
[296]	valid_0's l2: 0.357293
[297]	valid_0's l2: 0.357288
[298]	valid_0's l2: 0.357281
[299]	valid_0's l2: 0.357277
[300]	valid_0's l2: 0.35726
[301]	valid_0's l2: 0.357242
[302]	valid_0's l2: 0.357225
[303]	valid_0's l2: 0.357157
[304]	valid_0's l2: 0.357143
[305]	valid_0's l2: 0.35712
[306]	valid_0's l2: 0.357103
[307]	valid_0's l2: 0.357089
[308]	valid_0's l2: 0.357066
[309]	valid_0's l2: 0.357062
[310]	valid_0's l2: 0.357054
[311]	valid_0's l2: 0.357045
[312]	valid_0's l2: 0.357041
[313]	valid_0's l2: 0.357033
[314]	valid_0's l2: 0.35699
[315]	valid_0's l2: 0.356975
[316]	valid_0's l2: 0.356938
[317]	valid_0's l2: 0.356929
[318]	valid_0's l2: 0.356916
[319]	valid_0's l2: 0.356906
[320]	valid_0's l2: 0.356866
[321]	valid_0's l2: 0.356829
[322]	valid_0's l2: 0.356827
[323]	valid_0's l2: 0.356798
[324]	valid_0's l2: 0.356779
[325]	valid_0's l2: 0.356751
[326]	valid_0's l

[576]	valid_0's l2: 0.351405
[577]	valid_0's l2: 0.351368
[578]	valid_0's l2: 0.351359
[579]	valid_0's l2: 0.351353
[580]	valid_0's l2: 0.351336
[581]	valid_0's l2: 0.351325
[582]	valid_0's l2: 0.351304
[583]	valid_0's l2: 0.3513
[584]	valid_0's l2: 0.351295
[585]	valid_0's l2: 0.351286
[586]	valid_0's l2: 0.351286
[587]	valid_0's l2: 0.351284
[588]	valid_0's l2: 0.351278
[589]	valid_0's l2: 0.351271
[590]	valid_0's l2: 0.351267
[591]	valid_0's l2: 0.351224
[592]	valid_0's l2: 0.351215
[593]	valid_0's l2: 0.351204
[594]	valid_0's l2: 0.351199
[595]	valid_0's l2: 0.351192
[596]	valid_0's l2: 0.351185
[597]	valid_0's l2: 0.351174
[598]	valid_0's l2: 0.35116
[599]	valid_0's l2: 0.351155
[600]	valid_0's l2: 0.351155
[601]	valid_0's l2: 0.351131
[602]	valid_0's l2: 0.351131
[603]	valid_0's l2: 0.351103
[604]	valid_0's l2: 0.351097
[605]	valid_0's l2: 0.351073
[606]	valid_0's l2: 0.351051
[607]	valid_0's l2: 0.351028
[608]	valid_0's l2: 0.351026
[609]	valid_0's l2: 0.351023
[610]	valid_0's l

[861]	valid_0's l2: 0.346313
[862]	valid_0's l2: 0.346316
[863]	valid_0's l2: 0.346314
[864]	valid_0's l2: 0.34631
[865]	valid_0's l2: 0.346285
[866]	valid_0's l2: 0.346282
[867]	valid_0's l2: 0.346243
[868]	valid_0's l2: 0.346226
[869]	valid_0's l2: 0.346224
[870]	valid_0's l2: 0.346221
[871]	valid_0's l2: 0.346218
[872]	valid_0's l2: 0.346217
[873]	valid_0's l2: 0.346215
[874]	valid_0's l2: 0.346203
[875]	valid_0's l2: 0.346204
[876]	valid_0's l2: 0.346183
[877]	valid_0's l2: 0.346157
[878]	valid_0's l2: 0.346153
[879]	valid_0's l2: 0.346133
[880]	valid_0's l2: 0.346097
[881]	valid_0's l2: 0.346073
[882]	valid_0's l2: 0.346048
[883]	valid_0's l2: 0.346025
[884]	valid_0's l2: 0.346005
[885]	valid_0's l2: 0.345979
[886]	valid_0's l2: 0.345948
[887]	valid_0's l2: 0.345929
[888]	valid_0's l2: 0.345928
[889]	valid_0's l2: 0.345926
[890]	valid_0's l2: 0.345874
[891]	valid_0's l2: 0.345875
[892]	valid_0's l2: 0.345871
[893]	valid_0's l2: 0.345867
[894]	valid_0's l2: 0.345864
[895]	valid_0's

[1142]	valid_0's l2: 0.342589
[1143]	valid_0's l2: 0.342564
[1144]	valid_0's l2: 0.342552
[1145]	valid_0's l2: 0.342528
[1146]	valid_0's l2: 0.342507
[1147]	valid_0's l2: 0.342501
[1148]	valid_0's l2: 0.342485
[1149]	valid_0's l2: 0.342483
[1150]	valid_0's l2: 0.342475
[1151]	valid_0's l2: 0.342462
[1152]	valid_0's l2: 0.342451
[1153]	valid_0's l2: 0.34245
[1154]	valid_0's l2: 0.342443
[1155]	valid_0's l2: 0.342424
[1156]	valid_0's l2: 0.342403
[1157]	valid_0's l2: 0.342404
[1158]	valid_0's l2: 0.342402
[1159]	valid_0's l2: 0.342383
[1160]	valid_0's l2: 0.342364
[1161]	valid_0's l2: 0.342364
[1162]	valid_0's l2: 0.34236
[1163]	valid_0's l2: 0.342353
[1164]	valid_0's l2: 0.342347
[1165]	valid_0's l2: 0.342344
[1166]	valid_0's l2: 0.34234
[1167]	valid_0's l2: 0.34233
[1168]	valid_0's l2: 0.342326
[1169]	valid_0's l2: 0.342314
[1170]	valid_0's l2: 0.34228
[1171]	valid_0's l2: 0.342267
[1172]	valid_0's l2: 0.342265
[1173]	valid_0's l2: 0.342249
[1174]	valid_0's l2: 0.342249
[1175]	valid_0'

[1419]	valid_0's l2: 0.339603
[1420]	valid_0's l2: 0.339598
[1421]	valid_0's l2: 0.339586
[1422]	valid_0's l2: 0.339585
[1423]	valid_0's l2: 0.339581
[1424]	valid_0's l2: 0.339567
[1425]	valid_0's l2: 0.339552
[1426]	valid_0's l2: 0.339548
[1427]	valid_0's l2: 0.339537
[1428]	valid_0's l2: 0.339506
[1429]	valid_0's l2: 0.339505
[1430]	valid_0's l2: 0.339502
[1431]	valid_0's l2: 0.339492
[1432]	valid_0's l2: 0.339483
[1433]	valid_0's l2: 0.339472
[1434]	valid_0's l2: 0.339467
[1435]	valid_0's l2: 0.339468
[1436]	valid_0's l2: 0.339461
[1437]	valid_0's l2: 0.339418
[1438]	valid_0's l2: 0.339408
[1439]	valid_0's l2: 0.339409
[1440]	valid_0's l2: 0.339405
[1441]	valid_0's l2: 0.339405
[1442]	valid_0's l2: 0.339404
[1443]	valid_0's l2: 0.339396
[1444]	valid_0's l2: 0.339374
[1445]	valid_0's l2: 0.339368
[1446]	valid_0's l2: 0.339364
[1447]	valid_0's l2: 0.339357
[1448]	valid_0's l2: 0.339337
[1449]	valid_0's l2: 0.339329
[1450]	valid_0's l2: 0.339325
[1451]	valid_0's l2: 0.339323
[1452]	val

[1694]	valid_0's l2: 0.337502
[1695]	valid_0's l2: 0.337501
[1696]	valid_0's l2: 0.337492
[1697]	valid_0's l2: 0.337488
[1698]	valid_0's l2: 0.337482
[1699]	valid_0's l2: 0.337482
[1700]	valid_0's l2: 0.337477
[1701]	valid_0's l2: 0.337474
[1702]	valid_0's l2: 0.337467
[1703]	valid_0's l2: 0.337456
[1704]	valid_0's l2: 0.337421
[1705]	valid_0's l2: 0.337421
[1706]	valid_0's l2: 0.337415
[1707]	valid_0's l2: 0.337415
[1708]	valid_0's l2: 0.337413
[1709]	valid_0's l2: 0.337411
[1710]	valid_0's l2: 0.337409
[1711]	valid_0's l2: 0.337402
[1712]	valid_0's l2: 0.337402
[1713]	valid_0's l2: 0.337402
[1714]	valid_0's l2: 0.337393
[1715]	valid_0's l2: 0.337383
[1716]	valid_0's l2: 0.33738
[1717]	valid_0's l2: 0.337378
[1718]	valid_0's l2: 0.337378
[1719]	valid_0's l2: 0.337374
[1720]	valid_0's l2: 0.337364
[1721]	valid_0's l2: 0.337344
[1722]	valid_0's l2: 0.337341
[1723]	valid_0's l2: 0.337315
[1724]	valid_0's l2: 0.337311
[1725]	valid_0's l2: 0.337305
[1726]	valid_0's l2: 0.337285
[1727]	vali

In [41]:
y_pred = gbm.predict(X_valid, num_iteration=gbm.best_iteration)
#y_pred = np.argmax(y_pred, axis=1)

In [42]:
y_pred

array([ 2.31565148,  2.47586269,  2.37553958, ...,  1.19048797,
        1.33693581,  1.00767751])

# predict

In [51]:
test = pd.read_csv("~/environment/test.tsv", sep="\t")
events = pd.read_csv("~/environment/events.tsv", sep="\t")

In [52]:
print(users.shape)
users.head()

(69667, 5)


,user_id,age,gender,prefecture,created_on
0,1,34,女性,静岡県,2012-06-26
1,2,31,男性,鳥取県,2012-07-12
2,3,32,男性,東京都,2012-07-15
3,4,30,男性,東京都,2012-07-17
4,5,29,女性,埼玉県,2012-07-17


In [53]:
user_list = test["user_id"]
event_list = events["event_id"]

In [58]:
events.loc[:,"user_id"] = 1
user_data = pd.merge(events, users, on = "user_id")
data = data.rename(columns = {"prefecture_x": "event_prefercture", "prefecture_y":"user_prefecture"})

data.loc[:,"event_date"] = pd.to_datetime(data["event_start_at"])
data.loc[:,"event_year"] = data.loc[:,"event_date"].apply(lambda x:str(x.year))
data.loc[:,"event_month"] = data.loc[:,"event_date"].apply(lambda x:str(x.month))
data.loc[:,"event_day"] = data.loc[:,"event_date"].apply(lambda x:str(x.day))
data.loc[:,"event_weekday"] = data.loc[:,"event_date"].apply(lambda x:str(x.dayofweek))
data.loc[:,"event_hour"] = data.loc[:,"event_date"].apply(lambda x:str(x.hour))

data.loc[:,"user_date"] = pd.to_datetime(data["created_on"])
data.loc[:,"user_year"] = data.loc[:,"user_date"].apply(lambda x:str(x.year))
data.loc[:,"user_month"] = data.loc[:,"user_date"].apply(lambda x:str(x.month))

data.head()

,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,user_id_str,event_id_str,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,event_prefercture,first_published_at,female_price,male_price,interest,age,gender,user_prefecture,created_on,event_date,event_year,event_month,event_day,event_weekday,event_hour,user_date,user_year,user_month
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0,1,6261,27,39.0,27,39.0,2015-03-22 14:00:00,東京都,NaN,4000.0,6500.0,NaN,34,女性,静岡県,2012-06-26,2015-03-22 14:00:00,2015,3,22,6,14,2012-06-26,2012,6
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6
3,1,127600,2017-06-05 12:32:04,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6
4,1,127600,2017-06-06 09:37:45,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6


In [32]:
X = data[features]
print(X.shape)
X.head()

(1442610, 17)


,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_prefecture,female_price,male_price,age,gender,user_prefecture,event_year,event_month,event_day,event_weekday,event_hour,user_year,user_month
0,27,39.0,27,39.0,東京都,4000.0,6500.0,34,女性,静岡県,2015,3,22,6,14,2012,6
1,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6
2,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6
3,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6
4,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6


In [33]:
e_pre = prefecture_labeling(X["event_prefecture"])
e_pre.columns = "event_" + e_pre.columns.values
u_pre = prefecture_labeling(X["user_prefecture"])
u_pre.columns = "user_" + u_pre.columns.values
gen = gender_labeling(X["gender"])

temp = X[list(set(features) - set(["user_prefecture", "event_prefecture", "gender"]))]
print(temp.shape)
temp = pd.concat((temp,gen), axis = 1)
temp = pd.concat((temp,e_pre), axis = 1)
temp = pd.concat((temp,u_pre), axis = 1)

X = temp
print(X.shape)
X.head()

(1442610, 14)
(1442610, 17)


,male_age_lower,user_month,user_year,male_price,female_age_upper,female_age_lower,event_year,female_price,event_month,age,male_age_upper,event_day,event_weekday,event_hour,gender,event_prefecture,user_prefecture
0,27,6,2012,6500.0,39.0,27,2015,4000.0,3,34,39.0,22,6,14,0,27,43
1,26,6,2012,6000.0,36.0,24,2017,2000.0,6,34,38.0,11,6,10,0,8,43
2,26,6,2012,6000.0,36.0,24,2017,2000.0,6,34,38.0,11,6,10,0,8,43
3,26,6,2012,6000.0,36.0,24,2017,2000.0,6,34,38.0,11,6,10,0,8,43
4,26,6,2012,6000.0,36.0,24,2017,2000.0,6,34,38.0,11,6,10,0,8,43
